In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import pickle
TRAINING_DIR = "/content/drive/MyDrive/fire_dataset"
training_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip=True,
                                  rotation_range=30,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
VALIDATION_DIR = "Validation"
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = training_datagen.flow_from_directory("/content/drive/MyDrive/fire_dataset",
                                                       target_size=(224,224),class_mode='categorical',batch_size = 64)
validation_generator = validation_datagen.flow_from_directory(      
                                           "/content/drive/MyDrive/fire_dataset",
                                           target_size=(224,224),
                                           class_mode='categorical',
                                           batch_size= 16)

Found 999 images belonging to 2 classes.
Found 999 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
TRAINING_DIR = "/content/drive/MyDrive/FIRE-SMOKE-DATASET/Train"
training_datagen = ImageDataGenerator(rescale=1./255,
zoom_range=0.15,
horizontal_flip=True,
fill_mode='nearest')
VALIDATION_DIR = "/content/drive/MyDrive/FIRE-SMOKE-DATASET/Test"
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = training_datagen.flow_from_directory(
"/content/drive/MyDrive/FIRE-SMOKE-DATASET/Train",
target_size=(225,225),
shuffle = True,
class_mode='categorical',
batch_size = 128)
validation_generator = validation_datagen.flow_from_directory(
"/content/drive/MyDrive/FIRE-SMOKE-DATASET/Test",
target_size=(225,225),
class_mode='categorical',
shuffle = True,
batch_size= 14)


Found 2700 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


# New Section

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout

input_tensor = Input(shape=(224, 224, 3))

base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
  layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(
train_generator,
steps_per_epoch = 14,
epochs = 40,
validation_data = validation_generator,
validation_steps = 14)


Epoch 1/40
14/14 [==============================] - 193s 13s/step - loss: 14.7094 - acc: 0.5549 - val_loss: 0.4208 - val_acc: 0.8265
Epoch 2/40
14/14 [==============================] - 187s 13s/step - loss: 0.6228 - acc: 0.7828 - val_loss: 0.5345 - val_acc: 0.8061
Epoch 3/40
14/14 [==============================] - 187s 13s/step - loss: 0.5905 - acc: 0.8043 - val_loss: 0.3157 - val_acc: 0.8776
Epoch 4/40
14/14 [==============================] - 186s 13s/step - loss: 0.6258 - acc: 0.7751 - val_loss: 0.3568 - val_acc: 0.8827
Epoch 5/40
14/14 [==============================] - 187s 13s/step - loss: 0.3772 - acc: 0.8562 - val_loss: 1.1256 - val_acc: 0.6327
Epoch 6/40
14/14 [==============================] - 187s 13s/step - loss: 0.6130 - acc: 0.7936 - val_loss: 0.3763 - val_acc: 0.8673
Epoch 7/40
14/14 [==============================] - 197s 14s/step - loss: 0.3158 - acc: 0.8705 - val_loss: 0.5076 - val_acc: 0.7704
Epoch 8/40
14/14 [==============================] - 186s 13s/step - loss: 0

In [ ]:
history.history

{'loss': [9.361854553222656,
  0.5456726551055908,
  0.500946044921875,
  0.8191105723381042,
  0.41440731287002563,
  0.5133037567138672,
  0.3473086953163147,
  0.4398214817047119,
  0.25204211473464966,
  0.4378436803817749,
  0.3714744448661804,
  0.25903040170669556,
  0.3370296061038971,
  0.25852862000465393,
  0.26462700963020325,
  0.2426898330450058,
  0.2263115793466568,
  0.23513680696487427,
  0.2156522423028946,
  0.21706873178482056],
 'acc': [0.578125,
  0.7690930962562561,
  0.8253348469734192,
  0.767899751663208,
  0.8560267686843872,
  0.8060859441757202,
  0.8747016787528992,
  0.8400954604148865,
  0.9033412933349609,
  0.8532219529151917,
  0.8854415416717529,
  0.8967780470848083,
  0.8741050362586975,
  0.9079241156578064,
  0.8962053656578064,
  0.8997613191604614,
  0.9170644283294678,
  0.9110978245735168,
  0.9196428656578064,
  0.9206444025039673],
 'val_loss': [0.46736055612564087,
  1.1839420795440674,
  0.30300354957580566,
  0.29972636699676514,
  0.80

In [ ]:
#To train the top 2 inception blocks, freeze the first 249 layers and unfreeze the rest.
for layer in model.layers[:249]:
  layer.trainable = False
for layer in model.layers[249:]:
  layer.trainable = True
#Recompile the model for these modifications to take effect
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(
train_generator,
steps_per_epoch = 14,
epochs = 10,
validation_data = validation_generator,
validation_steps = 14)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
 9/14 [==================>...........] - ETA: 1:28 - loss: 0.8596 - acc: 0.7336

KeyboardInterrupt: ignored

In [ ]:
filename = "fire_pred_model.h5"
pickle.dump(model,open(filename,"wb"))



In [ ]:
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from keras.preprocessing import image
import pickle
import threading
from pydub import AudioSegment
from pydub.playback import play
from google.colab.patches import cv2_imshow


#Load the saved model

alarm_status =False
filename = "fire_pred_model.h5"
loaded_model = pickle.load(open(filename,"rb"))
video = cv2.VideoCapture("/content/drive/MyDrive/rain.mp4")
def play_alarm_sound_function():
	while True:
		playsound.playsound("/con",True)
while True:
        _, frame = video.read()
#Convert the captured frame into RGB
        im = Image.fromarray(frame, 'RGB')
#Resizing into 224x224 because we trained the model with this image size.
        im = im.resize((224,224))
        img_array = image.img_to_array(im)
        img_array = np.expand_dims(img_array, axis=0) / 255
        probabilities = model.predict(img_array)[0]
        #Calling the predict method on model to predict 'fire' on the image
        prediction = np.argmax(probabilities)
        #if prediction is 0, which means there is fire in the frame.
        if prediction == 0:
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
                print(probabilities[prediction])
                if (probabilities[prediction]) <= 0.85:
                  print("no fire detected")
                  
                else :
                  print("fire Detected  call the police" )
                if alarm_status == False :
                  song = AudioSegment.from_mp3("/content/drive/MyDrive/alarm-sound.mp3")
                  print('playing sound using  pydub')
                  play(song)
                  alarm_status = True

cv2.imshow("Capturing", frame)
while key==cv2.waitKey(1):
    if key == ord("q"):
        break
video.release()
cv2.destroyAllWindows()

0.4345704
no fire detected
playing sound using  pydub
0.5253097
no fire detected
0.5722328
no fire detected
0.6295514
no fire detected
0.5562957
no fire detected
0.6540123
no fire detected
0.4700249
no fire detected
0.5112041
no fire detected
0.5869414
no fire detected
0.51231205
no fire detected
0.51355106
no fire detected
0.4988029
no fire detected
0.48565978
no fire detected
0.5123414
no fire detected
0.54787606
no fire detected
0.6098409
no fire detected
0.5878043
no fire detected
0.54663515
no fire detected
0.47295228
no fire detected
0.50083894
no fire detected
0.5832646
no fire detected
0.68610996
no fire detected
0.55153114
no fire detected
0.6817989
no fire detected
0.59566945
no fire detected
0.63772315
no fire detected
0.60070163
no fire detected
0.5260485
no fire detected
0.58957124
no fire detected
0.8287521
no fire detected
0.8000783
no fire detected
0.67288846
no fire detected
0.65826243
no fire detected
0.6650449
no fire detected
0.77062106
no fire detected
0.5396042
no

AttributeError: ignored

In [ ]:

pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
